In [ ]:
import numpy as np
import scipy.stats as si
from statsmodels.tsa.seasonal import STL
import yfinance as yf
from datetime import datetime
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler

In [ ]:
cnf = {
    "start": "2020-01-01",
    "end": "2025-01-01",
    "ticker": "NVDA",
    "trend": None,
    "stl_period": 21,
    "robust": True,
    "market_ticker": "^GSPC",
    "rfr": 0.04,
    "w_delta": 21,
    "w_vol": 21,
    "delta_smooth_method": "mean",
    "gamma_pow_transform_method": "Identity",
    "vanna_log_transform_lambda": 1,
    "charm_tahn_sign_alpha": 0.5,
    "trend_power_transform_method": "Power"
}

In [ ]:
def get_data(ticker, start, end):
  data = yf.Ticker(ticker).history(start=start, end=end)
  return data.drop(["Dividends", "Stock Splits"], axis=1)

In [ ]:
data = get_data(ticker=cnf["ticker"], start=cnf["start"], end=cnf["end"])
market = get_data(ticker=cnf["market_ticker"], start=cnf["start"], end=cnf["end"])
vix = get_data(ticker="^VIX", start=cnf["start"], end=cnf["end"])["Close"]

In [ ]:
def apply_decomposition_metrics(log_returns: pd.DataFrame|np.ndarray,
                            period: int|None=None, trend: int|None=None,
                            robust: bool=True) -> pd.DataFrame:
  if not isinstance(log_returns, pd.DataFrame) and period is None:
    raise ValueError("If log_returns is not DataFrame, period should be specified")

  stl = STL(log_returns[1:,], period=period, robust=robust, trend=trend)
  res = stl.fit()

  return res.seasonal, res.trend

def compute_realized_vol(df: pd.DataFrame, lambda_: int|None=None, window: int|None=21):
  df = df.copy()
  df["LogRet"] = np.log(df['Close']).diff()
  if lambda_:
    df["RealizedVol"] = df["LogRet"].emw((1-lambda_)).std() * np.sqrt(252)

  elif window:
    df["RealizedVol"] = df["LogRet"].rolling(window).std() * np.sqrt(252)

  return df

def get_estimation(x, y, window):
    x = x.fillna(0)
    deltas, gammas, vannas, vegas, index = [], [], [], [], []

    for i in range(window, len(y)+1):
        x_window = x.iloc[i-window:i]
        y_window = y.iloc[i-window:i]

        model = sm.OLS(y_window, sm.add_constant(x_window)).fit()

        deltas.append(model.params['mkt'])
        gammas.append(2 * model.params['mkt_sq'])
        vannas.append(model.params['mkt_interaction'])
        vegas.append(model.params['vix'])
        index.append(y.index[i-1])

    return (
        pd.Series(deltas, index=index),
        pd.Series(gammas, index=index),
        pd.Series(vannas, index=index),
        pd.Series(vegas, index=index),
    )

def get_sigma_hat(r_i: pd.Series, window: int=21):
  return r_i.ewm(span=window).std()

def get_charm(deltas: list, L:int=5):
  charm = deltas.rolling(L).mean()
  return charm

def apply_greeks(market_returns: pd.DataFrame, stock_returns: pd.DataFrame,
                 vix: pd.DataFrame, sigma: pd.DataFrame, window: int= 21, L:int=5):
    sigma_diff = sigma.diff().fillna(0)
    X = pd.DataFrame({
        'mkt': market_returns,
        'mkt_sq': market_returns**2,
        'mkt_interaction': market_returns * sigma_diff,
        "vix": vix
    }, index=stock_returns.index)

    y = stock_returns
    delta, gamma, vanna, vega = get_estimation(window=window, x=X, y=y)
    charm = get_charm(delta)
    return delta, gamma, vanna, vega, charm

def apply_scaler(df:pd.DataFrame):
  scaler = RobustScaler()
  columns = df.columns
  df_scaled = scaler.fit_transform(df)

  for k, v in zip(columns, df_scaled.T):
    df[k] = v

  return df

def apply_inverse_logistic_penalty(feature, a, window, method):
  if method == "mean":
    t = feature.rolling(window=window, min_periods=1).mean()
  elif method == "median":
    t = feature.rolling(window=window, min_periods=1).median()
  else:
    t = 0

  z = a*(feature-t)
  return 1/(1+np.exp(z))

def apply_power_transform(feature:pd.Series, method:str="Identity", power:float|int=2):
  x = np.array(feature, dtype=float)

  if method == "Identity":
    return x
  elif method == "Sqrt":
    return np.sqrt(np.maximum(x, 0))
  elif method == "Power":
    return np.power(x, power)
  elif method == "Reciprocal":
    return 1 / (x + 1e-8)
  elif method == "Log":
    return np.log(x + 1e-8)
  else:
    raise ValueError(f"Unknown transform method {method}")

def apply_tahn_transform(feature: pd.Series, alpha:float=0.5):
  return np.sign(feature) * np.tahn(alpha * np.abs(feature))

def apply_log_transform(feature: pd.Series, l:int=1):
  x = np.array(feature, dtype=float)
  return np.log(1+l*x)

def apply_softplus(feature: pd.Series):
  return np.log(1 + np.exp(feature))

def scale(df: pd.DataFrame, a:int=1, window:int=5, method: str|None=None,
          l_gamma:str="Identity", l_vanna:int=1, tahn_sign_alpha:float=0.5, trend_method:str="Power"):
  df_scaled = apply_scaler(df=df)

  df_scaled['Delta'] = apply_inverse_logistic_penalty(feature=df_scaled['Delta'], a=a, window=window, method=method)
  df_scaled['Gamma'] = apply_power_transform(feature=df_scaled['Gamma'], method=l_gamma)
  df_scaled['Vanna'] = apply_log_transform(feature=df_scaled['Vanna'], l=l_vanna)
  df_scaled['Vega'] = apply_log_transform(feature=df_scaled['Vega'], l=l_vanna)
  df_scaled['Charm'] = apply_tahn_transform(feature=df_scaled['Charm'], alpha=tahn_sign_alpha)
  df_scaled['Season'] = apply_softplus(feature=df_scaled['Season'])
  df_scaled['Trend'] = apply_power_transform(feature=df_scaled['Trend'], method=trend_method)

In [ ]:
market = compute_realized_vol(market)
data = compute_realized_vol(data)
season, trend  = apply_decomposition_metrics(data["LogRet"], period=cnf["stl_period"])

df = pd.DataFrame({
    "Trend": trend,
    "Season": season
}, index=data.index)

sigma_hat = get_sigma_hat(market["LogRet"], window=cnf['w_vol'])

greeks = apply_greeks(
    market_returns=market["LogRet"], stock_returns=data["LogRet"],
    vix=vix, sigma=sigma_hat, window=cnf['w_delta']
)
for k, v in zip(["Delta", "Gamma", "Vanna", "Vega", "Charm"], greeks):
  df[k] = v

df = scale(df=df, method=cnf["delta_smooth_method"],
           l_gamma=cnf["gamma_pow_transform_method"],
           l_vanna=cnf["vanna_log_transform_lambda"],
           tahn_sign_alpha=cnf["charm_tahn_sign_alpha"],
           trend_method = cnf["trend_power_transform_method"]
)
